In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd

import PyPDF2

Importing the Document

In [10]:
document_path = "./Looking Back on High School Life - Hikigaya Hachiman.pdf"

In [11]:
document = open(document_path, 'rb')

In [12]:
pdfReader = PyPDF2.PdfReader(document)

num_pages = len(pdfReader.pages)

text = ""

for page in range(num_pages):
    pageHandle = pdfReader.pages[page]
    text += pageHandle.extract_text()

In [13]:
document.close()

In [14]:
text

"'Looking Back on High School Life'\nClass 2F – Hikigaya Hachiman\nYouth is a lie. It is nothing but evil.\nThose of you who rejoice in youth are perpetually deceiving yourselves and those around you. You \nperceive everything about the reality surrounding you in a positive light. Even life-threatening \nmistakes will be remembered as single page proofs of your youth.\nI'll give you an example. If such people were to dabble in criminal acts such as shoplifting or mass \nrioting, it would be called ‘youthful indiscretion.’ If they were to fail an exam, they would say \nschool is not only a place for studying. Their pursuit of 'youth' excuses even distortions of \ncommonly held beliefs and social norms.\nUnder their discretion, lies, secrets, crimes and even failure are nothing but the spice of one's youth. \nAnd in their corrupt ways, they discover something peculiar about failure. They conclude that while  \ntheir own failures are generally a part of relishing in youth, others' failure

In [ ]:
!pip install transformers torch

Text Chunking

In [16]:
def chunkText(text, max_length = 512):
    chunks = []
    curr_chunk = []

    words = text.split()

    curr_length = 0
    for word in words:

        if curr_length + len(word) + 1 > max_length:
            chunks.append(' '.join(curr_chunk))
            curr_chunk = [word]
            curr_length = len(word)
        else:
            curr_chunk.append(word)
            curr_length += len(word) + 1

    if curr_chunk:
        chunks.append(' '.join(curr_chunk))

    return chunks

In [19]:
from transformers import pipeline

Initializing the Pipeline

In [20]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Analyzing Chunks

In [22]:
chunks = chunkText(text)
sentiments = []

for chunk in chunks:
    result = sentiment_pipeline(chunk)[0]
    sentiments.append(result)

Overall Scores

In [23]:
positive_score = sum(s['score'] for s in sentiments if s['label'] == 'POSITIVE')
negative_score = sum(s['score'] for s in sentiments if s['label'] == 'NEGATIVE')
total_chunks = len(sentiments)

In [24]:
print(f"Positive sentiment: {positive_score / total_chunks:.2f}")
print(f"Negative sentiment: {negative_score / total_chunks:.2f}")

Positive sentiment: 0.33
Negative sentiment: 0.53


Scores by Chunk

In [25]:
for i, sentiment in enumerate(sentiments):
    print(f"Chunk {i+1}: {sentiment['label']} (Score: {sentiment['score']:.2f})")

Chunk 1: POSITIVE (Score: 1.00)
Chunk 2: NEGATIVE (Score: 0.61)
Chunk 3: NEGATIVE (Score: 1.00)
